In [1]:
# Dependencies
import pandas as pd
import numpy as np
from collections import defaultdict
import re

In [2]:
# Store filepath in a variable
data = pd.read_csv("Resources/all_data.csv", encoding="ISO-8859-1")
data.head()

,ï»¿RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,PVOTVLCD_END,PVOTVLCD_DURATION,HesPauseAmb,HesPauseAmb_START,HesPauseAmb_END,HesPauseAmb_DURATION,NOTES,Notes_START,Notes_END,Notes_DURATION
0,DS,rPPA,15,WAB Naming,1,Baseline,NaN,NaN,NaN,NaN,...,NaN,NaN,Art,1900.048723,1900.200674,0.151950,NaN,0.000000,1900.200674,1900.200674
1,DS,rPPA,15,WAB Naming,1,Baseline,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1900.200674,1900.496319,0.295646,Article (É) before word,1900.200674,1900.496319,0.295646
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1900.225036,0.024363,NaN,NaN,NaN,NaN,Article (É) before word,1900.200674,1900.496319,0.295646
3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,NaN,NaN,NaN,NaN,NaN,NaN,Article (É) before word,1900.200674,1900.496319,0.295646
4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1900.424612,0.036167,NaN,NaN,NaN,NaN,Article (É) before word,1900.200674,1900.496319,0.295646


In [3]:
# Create a subset of data based on data you are interested in working with
data = data[
    ['ï»¿RA', 
    'Project', 
    'PID', 
    'Test', 
    'Arm', 
    'Week', 
    'Target ',
    ' Production ', 
    ' PRODUCTION_START ', 
    ' PRODUCTION_END ',
    ' PRODUCTION_DURATION ', 
    ' Phoneme ', ' PHONEME_START ',
    ' PHONEME_END ', 
    ' PHONEME_DURATION ',
    ' NOTES ']
       ]

# Rename the columns to something easier to work with
data.rename(
    columns={
       'ï»¿RA':'RA', 
       'Target ':'Target',
       ' Production ':'Production', 
       ' PRODUCTION_START ':'PRODUCTION_START', 
       ' PRODUCTION_END ':'PRODUCTION_END',
       ' PRODUCTION_DURATION ':'PRODUCTION_DURATION', 
       ' Phoneme ':'Phoneme', 
       ' PHONEME_START ':'PHONEME_START',
       ' PHONEME_END ':'PHONEME_END', 
       ' PHONEME_DURATION ':'PHONEME_DURATION',
       ' NOTES ':'NOTES'
       }, inplace=True)

# Drop duplicates and NaNs
data = data.dropna(subset=['Target', 'Phoneme']).drop_duplicates()

# Check data
data.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,Phoneme,PHONEME_START,PHONEME_END,PHONEME_DURATION,NOTES
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,B,1900.200674,1900.225036,0.024363,Article (É) before word
3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,UH,1900.225036,1900.388444,0.163408,Article (É) before word
4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,K,1900.388444,1900.496319,0.107875,Article (É) before word
8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,B,1905.496713,1905.51091,0.014197,NaN
9,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,AO,1905.510910,1905.721917,0.211006,NaN


In [4]:
#Check that target words are correctly written
data['Target'].unique()

array(['book', 'ball', 'knife', 'cup', 'safety pin', 'hammer',
       'toothbrush', 'eraser', 'lock', 'pencil', 'screwdriver', 'key',
       'paper clip', 'watch', 'comb', 'rubber band', 'spoon', 'tape',
       'fork', 'matches', 'toothpaste', 'rubber', 'paper', 'S UH Q IH',
       'ball '], dtype=object)

In [5]:
#Replace incorrect target words
data['Target'] = (
    data['Target']
    .replace(
        {
        'rubber':'rubber band', 
        'paper':'paper clip',
        'ball ':'ball',
        'toothpaste':'toothbrush'
        }
    )
)

#Get rid of the one that couldn't be matched to a specific target
data = data.loc[data["Target"] != 'S UH Q IH']

#Check to see if they all look good now
data['Target'].unique()

array(['book', 'ball', 'knife', 'cup', 'safety pin', 'hammer',
       'toothbrush', 'eraser', 'lock', 'pencil', 'screwdriver', 'key',
       'paper clip', 'watch', 'comb', 'rubber band', 'spoon', 'tape',
       'fork', 'matches'], dtype=object)

In [6]:
#Create function to assign a unique ID to words
word_id = defaultdict(lambda: len(word_id))
data['Word_ID'] = [word_id[i]+1 for i in data['Target']]

data.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,Phoneme,PHONEME_START,PHONEME_END,PHONEME_DURATION,NOTES,Word_ID
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,B,1900.200674,1900.225036,0.024363,Article (É) before word,1
3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,UH,1900.225036,1900.388444,0.163408,Article (É) before word,1
4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,K,1900.388444,1900.496319,0.107875,Article (É) before word,1
8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,B,1905.496713,1905.51091,0.014197,NaN,2
9,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,AO,1905.510910,1905.721917,0.211006,NaN,2


In [7]:
#Assign a numeric ID to each session

data2=data

data2['Session_ID'] = data2["Arm"].astype(str) + data2["Week"].astype(str)

data2['Session_ID'] = (
    data2['Session_ID']
    .replace(
        {
        '1Baseline':'0', 
        '112':'1',
        '212':'2'
        }
    )
)
data2['Session_ID'].unique()

array(['0', '1', '2'], dtype=object)

In [8]:
#Assign production number to label when each production attempt was made for each word
data2['Prod_Word_N'] = data2.groupby(['PID','Session_ID'])['PRODUCTION_START'].rank('dense').astype(int)
data2.reset_index(inplace=True)
data2.head()

,index,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,Phoneme,PHONEME_START,PHONEME_END,PHONEME_DURATION,NOTES,Word_ID,Session_ID,Prod_Word_N
0,2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,B,1900.200674,1900.225036,0.024363,Article (É) before word,1,0,1
1,3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,UH,1900.225036,1900.388444,0.163408,Article (É) before word,1,0,1
2,4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,K,1900.388444,1900.496319,0.107875,Article (É) before word,1,0,1
3,8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,B,1905.496713,1905.51091,0.014197,NaN,2,0,2
4,9,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,AO,1905.510910,1905.721917,0.211006,NaN,2,0,2


In [9]:
#Assign production number to label when each production attempt was made for each phoneme
data2['Prod_Phon_N'] = data2.groupby(['PID','Session_ID','PRODUCTION_START'])['PHONEME_START'].rank('dense').astype(int)
data2.reset_index(inplace=True)
data2.head()

,level_0,index,RA,Project,PID,Test,Arm,Week,Target,Production,...,PRODUCTION_DURATION,Phoneme,PHONEME_START,PHONEME_END,PHONEME_DURATION,NOTES,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N
0,0,2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,0.295646,B,1900.200674,1900.225036,0.024363,Article (É) before word,1,0,1,1
1,1,3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,0.295646,UH,1900.225036,1900.388444,0.163408,Article (É) before word,1,0,1,2
2,2,4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,0.295646,K,1900.388444,1900.496319,0.107875,Article (É) before word,1,0,1,3
3,3,8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,0.397365,B,1905.496713,1905.51091,0.014197,NaN,2,0,2,1
4,4,9,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,0.397365,AO,1905.510910,1905.721917,0.211006,NaN,2,0,2,2


In [10]:
#Create a code to tie each word and phoneme together based on phoneme position
data2['Code'] = data2["Word_ID"].astype(str) + str('_') + data2["Prod_Phon_N"].astype(str)

#Create a separate code to tie each participant to their word-phoneme production pairs
data2['Phon_Sess_Code'] = (
    data2["PID"].astype(str) + 
    str('_') + 
    data2["Session_ID"].astype(str) + 
    str('_') + 
    data2["Prod_Word_N"].astype(str) +
    str('_') + 
    data2["Prod_Phon_N"].astype(str)
    .astype(str)
)

#Create a separate code to tie each participant to their word productions
data2['Word_Sess_Code'] = (
    data2["PID"].astype(str) + 
    str('_') + 
    data2["Session_ID"].astype(str) + 
    str('_') + 
    data2["Prod_Word_N"].astype(str)
    .astype(str)
)

data2.head()

,level_0,index,RA,Project,PID,Test,Arm,Week,Target,Production,...,PHONEME_END,PHONEME_DURATION,NOTES,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code
0,0,2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,1900.225036,0.024363,Article (É) before word,1,0,1,1,1_1,15_0_1_1,15_0_1
1,1,3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,1900.388444,0.163408,Article (É) before word,1,0,1,2,1_2,15_0_1_2,15_0_1
2,2,4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,1900.496319,0.107875,Article (É) before word,1,0,1,3,1_3,15_0_1_3,15_0_1
3,3,8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,1905.51091,0.014197,NaN,2,0,2,1,2_1,15_0_2_1,15_0_2
4,4,9,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,1905.721917,0.211006,NaN,2,0,2,2,2_2,15_0_2_2,15_0_2


In [11]:
#Before we can do anything with the syllables we need to identify the last phoneme produced within each production attempt

last_prod_phon = data2.groupby('Word_Sess_Code')['Prod_Phon_N'].last().to_frame()

last_prod_phon['Prod_Last_Phon'] = 1

last_prod_phon = last_prod_phon.reset_index()

last_prod_phon['Phon_Sess_Code'] = (
    last_prod_phon["Word_Sess_Code"].astype(str) + 
    str('_') + 
    last_prod_phon["Prod_Phon_N"].astype(str)
)

last_prod_phon = last_prod_phon.drop(columns=['Word_Sess_Code','Prod_Phon_N'])

last_prod_phon

,Prod_Last_Phon,Phon_Sess_Code
0,1,12_0_1_3
1,1,12_0_10_5
2,1,12_0_11_5
3,1,12_0_12_13
4,1,12_0_13_2
...,...,...
571,1,8_2_5_3
572,1,8_2_6_8
573,1,8_2_7_4
574,1,8_2_8_7


In [12]:
#Add the identification for the last phoneme produced to the main dataset
data2 = pd.merge(data2, last_prod_phon, on='Phon_Sess_Code', how='left')
data2['Prod_Last_Phon'] = data2['Prod_Last_Phon'].fillna(0).astype('int')
data2.head(5)

,level_0,index,RA,Project,PID,Test,Arm,Week,Target,Production,...,PHONEME_DURATION,NOTES,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon
0,0,2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,0.024363,Article (É) before word,1,0,1,1,1_1,15_0_1_1,15_0_1,0
1,1,3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,0.163408,Article (É) before word,1,0,1,2,1_2,15_0_1_2,15_0_1,0
2,2,4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,0.107875,Article (É) before word,1,0,1,3,1_3,15_0_1_3,15_0_1,1
3,3,8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,0.014197,NaN,2,0,2,1,2_1,15_0_2_1,15_0_2,0
4,4,9,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,0.211006,NaN,2,0,2,2,2_2,15_0_2_2,15_0_2,0


In [13]:
#Check what each phoneme is being registered as
results = (
    # trans is a series, so use string accessor to split value strings
    data2["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
)

results.unique()

array(['B', '', 'UH', 'K', 'AO', 'L', 'SH', 'N', 'AY', 'F', 'AH', 'P',
       'S', '.', 'T', 'IY', 'H', 'AE', 'M', 'ER', 'UX', 'OW', 'UY', 'TH',
       'R', 'AXR', 'AA', 'EH', 'AX', 'UW', 'DX', 'CH', 'ZH', 'IH', 'EY',
       'W', 'Q', 'EI', 'SEYF', 'HH', 'JH', 'D', 'NG', 'G', 'Z', 'V', 'U',
       'AWR', 'E', 'SK', 'LIH', 'OA', 'A'], dtype=object)

In [14]:
#See all rows for alone 'U' items to get string environment for replacement
p = '\sU\s'
data2[data2['Production'].str.contains(p)]

,level_0,index,RA,Project,PID,Test,Arm,Week,Target,Production,...,PHONEME_DURATION,NOTES,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon
1286,1286,3275,GO,rPPA,1,WAB Naming,2,12,screwdriver,S K R U . JH AXR,...,0.110428,Sounds like screwdr-,11,2,28,1,11_1,1_2_28_1,1_2_28,0
1287,1287,3276,GO,rPPA,1,WAB Naming,2,12,screwdriver,S K R U . JH AXR,...,0.119282,Sounds like screwdr-,11,2,28,2,11_2,1_2_28_2,1_2_28,0
1288,1288,3278,GO,rPPA,1,WAB Naming,2,12,screwdriver,S K R U . JH AXR,...,0.037095,Sounds like screwdr-,11,2,28,3,11_3,1_2_28_3,1_2_28,0
1289,1289,3279,GO,rPPA,1,WAB Naming,2,12,screwdriver,S K R U . JH AXR,...,0.059138,Sounds like screwdr-,11,2,28,4,11_4,1_2_28_4,1_2_28,0
1290,1290,3280,GO,rPPA,1,WAB Naming,2,12,screwdriver,S K R U . JH AXR,...,0.049474,Sounds like screwdr-,11,2,28,5,11_5,1_2_28_5,1_2_28,0
1291,1291,3281,GO,rPPA,1,WAB Naming,2,12,screwdriver,S K R U . JH AXR,...,0.128895,Sounds like screwdr-,11,2,28,6,11_6,1_2_28_6,1_2_28,1


In [15]:
#See all rows for alone 'E' items to get string environment for replacement
p = '\sE\s'
data2[data2['Production'].str.contains(p)]

,level_0,index,RA,Project,PID,Test,Arm,Week,Target,Production,...,PHONEME_DURATION,NOTES,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon
1367,1367,3534,GO,rPPA,1,WAB Naming,2,12,matches,M E T,...,0.106330,"After clinician says ""start with m-"" M",20,2,55,1,20_1,1_2_55_1,1_2_55,0
1368,1368,3535,GO,rPPA,1,WAB Naming,2,12,matches,M E T,...,0.127582,"After clinician says ""start with m-"" M",20,2,55,2,20_2,1_2_55_2,1_2_55,0
1369,1369,3536,GO,rPPA,1,WAB Naming,2,12,matches,M E T,...,0.135479,"After clinician says ""start with m-"" M",20,2,55,3,20_3,1_2_55_3,1_2_55,1


In [16]:
#See all rows for alone 'A' items to get string environment for replacement
p = '\sA\s'
data2[data2['Production'].str.contains(p)]

,level_0,index,RA,Project,PID,Test,Arm,Week,Target,Production,...,PHONEME_DURATION,NOTES,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon
1848,1848,4387,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.087184,NaN,16,2,18,1,16_1,12_2_18_1,12_2_18,0
1849,1849,4388,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.099706,NaN,16,2,18,2,16_2,12_2_18_2,12_2_18,0
1850,1850,4389,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.034134,NaN,16,2,18,3,16_3,12_2_18_3,12_2_18,0
1851,1851,4391,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.090724,NaN,16,2,18,4,16_4,12_2_18_4,12_2_18,0
1852,1852,4392,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.064674,NaN,16,2,18,5,16_5,12_2_18_5,12_2_18,0
1853,1853,4394,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.143721,NaN,16,2,18,6,16_6,12_2_18_6,12_2_18,0
1854,1854,4395,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.051201,NaN,16,2,18,7,16_7,12_2_18_7,12_2_18,0
1855,1855,4396,MZ,rPPA,12,WAB Naming,2,12,rubber band,R UH . B ER . B A N D,...,0.089579,NaN,16,2,18,8,16_8,12_2_18_8,12_2_18,1


In [17]:
# Replace incorrent substrings (pt. 1)
dat = data2
dat['Production'] = (
    dat['Production']
    .replace(
            {
            '\.':'',
            'SEYF':'S EY F',
            'OA':'AO',
            'HH':'H',
            'LIH':'L IH',
            'AWR':'AXR',
            'SK':'S K',
            'UX':'UW',
            'UY':'UW',
            'R\sU\s':'R UW ', #Needed to write this one separately to not replace all of the 'U's in other ARPABET sequences
            'M E T':'M EH T', #Needed to write this one separately to not replace all of the 'E's in other ARPABET sequences
            'B\sA\s':'B AE ', #Needed to write this one separately to not replace all of the 'A's in other ARPABET sequences
            '\s\s|\s\s\s':' ',
            'EI':'EY'
            },
        regex=True)
    .str.strip()
)

#For whatever reason, this needed to be run twice to get rid of the ''

In [18]:
# Replace incorrent substrings (pt. 2)
dat = data2
dat['Production'] = (
    dat['Production']
    .replace(
            {
            '\.':'',
            'SEYF':'S EY F',
            'OA':'AO',
            'HH':'H',
            'LIH':'L IH',
            'AWR':'AXR',
            'SK':'S K',
            'UX':'UW',
            'UY':'UW',
            'R\sU\s':'R UW ', #Needed to write this one separately to not replace all of the 'U's in other ARPABET sequences
            'M E T':'M EH T', #Needed to write this one separately to not replace all of the 'E's in other ARPABET sequences
            'B\sA\s':'B AE ', #Needed to write this one separately to not replace all of the 'A's in other ARPABET sequences
            '\s\s|\s\s\s':' ',
            'EI':'EY'
            },
        regex=True)
    .str.strip()
)

#For whatever reason, this needed to be run twice to get rid of the ''

In [19]:
#Recheck what each phoneme is being registered as
results = (
    # trans is a series, so use string accessor to split value strings
    dat["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
)

results.unique()

array(['B', 'UH', 'K', 'AO', 'L', 'SH', 'N', 'AY', 'F', 'AH', 'P', 'S',
       'T', 'IY', 'H', 'AE', 'M', 'ER', 'UW', 'OW', 'TH', 'R', 'AXR',
       'AA', 'EH', 'AX', 'DX', 'CH', 'ZH', 'IH', 'EY', 'W', 'Q', 'JH',
       'D', 'NG', 'G', 'Z', 'V'], dtype=object)

In [20]:
#Some of the words still have some extra spaces (determined by error message when trying to convert the productions to IPA), so going to delete all spaces and then add them back in

dat["Production"] = (
    # Production is a series, so use string accessor to split value strings
    dat["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
    # Remove blank
    .dropna()
    # group them by the original index
    .groupby(level=0)
    # aggregate them with a space
    .agg(lambda s: " ".join(s))
)

dat["Production"].unique()

array(['B UH K', 'B AO L', 'SH', 'N', 'N AY F', 'K AH P', 'S',
       'S AY F T IY', 'H AE M ER', 'T UW SH B OW N', 'T UW', 'B UH SH',
       'T UH', 'T IY', 'T UW TH P IY S', 'T P IY', 'T IY S', 'P IY', 'P',
       'IY', 'R AH B', 'R AH B AXR N', 'R AH B AXR', 'L AA K AXR',
       'T EH M', 'T', 'P EH N S AX L', 'S T R UW', 'S T R',
       'S T R UW DX OW', 'S K R UW', 'S K R UW CH', 'S K R UW S', 'K',
       'ZH', 'K IY', 'T S', 'S IH M', 'P EY P AXR', 'W AA CH', 'OW',
       'R AH B AXR L', 'R AH B AXR N AX Q', 'R AH', 'K AH', 'Q', 'S P UW',
       'S P UH SH', 'S P', 'S B AH', 'S UW P', 'T EY', 'SH AO R', 'B AX',
       'AY F', 'AY', 'K AO', 'K AH F', 'K AO Q', 'K AO F', 'K AH Q',
       'S T EY Q', 'S T EY', 'S T AY', 'H', 'H AE', 'H AE M AXR',
       'T IH TH', 'P EY S', 'B AA', 'P AH', 'B', 'B AH', 'L', 'L AA K',
       'P EH N CH R', 'P EH N S IH', 'P EH N', 'P EH', 'P IH N', 'K OW',
       'K AX', 'S T IH F', 'S P UW N', 'T AE', 'P AX', 'P IH', 'P T S',
       'F AX', 'F', 'S 

In [21]:
#Translate the ARPABET codes to IPA codes
dictionary = (
    pd.read_csv("Resources/dict.csv")
    .set_index("Arpabet")
)

dat["Production_IPA"] = (
    # The production column is a series, so use string accessor to split value strings
    dat["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
    # perform the lookup in the dictionary of each individual value
    .apply(lambda v: dictionary.loc[v])
    # group them by the original index
    .groupby(level=0)
    # "sum" them, which for string, concatonates them without any spaces
    .sum()
)

dat.head()

,level_0,index,RA,Project,PID,Test,Arm,Week,Target,Production,...,NOTES,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA
0,0,2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,Article (É) before word,1,0,1,1,1_1,15_0_1_1,15_0_1,0,bʊk
1,1,3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,Article (É) before word,1,0,1,2,1_2,15_0_1_2,15_0_1,0,bʊk
2,2,4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,...,Article (É) before word,1,0,1,3,1_3,15_0_1_3,15_0_1,1,bʊk
3,3,8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,NaN,2,0,2,1,2_1,15_0_2_1,15_0_2,0,bɔl
4,4,9,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,...,NaN,2,0,2,2,2_2,15_0_2_2,15_0_2,0,bɔl


In [22]:
#Import phonetic feature identifies
phon_dist_features = (
    pd.read_csv("Resources/phon_dist_features.csv")
)
phon_dist_features = phon_dist_features.dropna()
phon_dist_features['Phoneme_ID'] = phon_dist_features['Phoneme_ID'].astype('int')

In [23]:
#Create a dictionary for phoneme ID number

phon_ID = phon_dist_features[['IPA_singles','Arpabet','Phoneme_ID']]

phon_ID.rename(
    columns={
       'IPA_singles':'IPA_singles', 
       'Arpabet':'Phoneme'
       }, inplace=True)

phon_ID

/var/folders/b9/y7yy3gjx1vjcj8dmj6zw15kw0000gn/T/ipykernel_12356/3682838066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phon_ID.rename(


,IPA_singles,Phoneme,Phoneme_ID
0,h,H,1
1,r,R,2
2,w,W,3
3,j,Y,4
4,b,B,5
5,ʧ,CH,6
6,d,D,7
7,ð,DH,8
8,ɾ,DX,9
9,f,F,10


In [24]:
#Merge the dictionary to the main dataset on Phoneme, so that corresponding IPA symbols are added as a column
df3 = pd.merge(dat, phon_ID, on='Phoneme', how='left')
df3 = df3.drop(columns=['level_0','index']) #drop unneeded columns
df3.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,1,1_1,15_0_1_1,15_0_1,0,bʊk,b,5.0
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,2,1_2,15_0_1_2,15_0_1,0,bʊk,ʊ,36.0
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,3,1_3,15_0_1_3,15_0_1,1,bʊk,k,13.0
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,0,2,1,2_1,15_0_2_1,15_0_2,0,bɔl,b,5.0
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,0,2,2,2_2,15_0_2_2,15_0_2,0,bɔl,ɔ,30.0


In [25]:
#Still some problems with how certain phonemes were coded (determined by error message), going to find them and put them into their own dataset
na_df = df3[df3['Phoneme_ID'].isna()] 

# Export fix it file as a CSV, without the Pandas index, but with the header
na_df.to_csv("Resources/FixThese.csv", index=False, header=True)
na_df.columns.to_list()

['RA',
 'Project',
 'PID',
 'Test',
 'Arm',
 'Week',
 'Target',
 'Production',
 'PRODUCTION_START',
 'PRODUCTION_END',
 'PRODUCTION_DURATION',
 'Phoneme',
 'PHONEME_START',
 'PHONEME_END',
 'PHONEME_DURATION',
 'NOTES',
 'Word_ID',
 'Session_ID',
 'Prod_Word_N',
 'Prod_Phon_N',
 'Code',
 'Phon_Sess_Code',
 'Word_Sess_Code',
 'Prod_Last_Phon',
 'Production_IPA',
 'IPA_singles',
 'Phoneme_ID']

In [26]:
#Check what phoneme codes are present in the dataset
dat['Phoneme'].unique()

array(['B', 'UH', 'K', 'AO', 'L', 'SH', 'N', 'AY', 'F', 'AH', 'P', 'S',
       'T', 'IY', 'H', 'AE', 'M', 'ER', 'UX', 'OW', 'UY', 'TH', 'R',
       'AXR', 'AA', 'EH', 'AX', 'UW', 'DX', 'CH', 'ZH', 'IH', 'EY', 'W',
       'Q', 'EI', ' EY', 'HH', 'JH', 'K L IH P', 'D', 'NG', 'G', ' M',
       'Z', 'S ', 'EH ', 'V', 'T EY K', 'M ', 'AWR', 'K ', 'E', 'c', 'SK',
       'LIH', 'OA', 'Ì\x81TH'], dtype=object)

In [27]:
#Replace problem phonemes with correct ARPABET code or to a more readable format
dat['Phoneme'] = (
    dat['Phoneme']
    .replace(
            {
            'Ì\x81TH':'TH',
            'c':'K',
            'OA':'AO',
            'HH':'H',
            'AWR':'AXR',
            'LIH':'L IH',
            'SK':'S K',
            'UX':'UW',
            'UY':'UW',
            'EI':'EY'
            },
        regex=True)
    .str.strip() #gets rid of white space at the beginning and ending of string
)
dat['Phoneme'].unique()

array(['B', 'UH', 'K', 'AO', 'L', 'SH', 'N', 'AY', 'F', 'AH', 'P', 'S',
       'T', 'IY', 'H', 'AE', 'M', 'ER', 'UW', 'OW', 'TH', 'R', 'AXR',
       'AA', 'EH', 'AX', 'DX', 'CH', 'ZH', 'IH', 'EY', 'W', 'Q', 'JH',
       'K L IH P', 'D', 'NG', 'G', 'Z', 'V', 'T EY K', 'E', 'S K', 'L IH'],
      dtype=object)

In [28]:
#Merge the phonemes again to get the correct IPA symbols
df3 = pd.merge(dat, phon_ID, on='Phoneme', how='left')
df3 = df3.drop(columns=['level_0','index'])
df3.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,1,1_1,15_0_1_1,15_0_1,0,bʊk,b,5.0
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,2,1_2,15_0_1_2,15_0_1,0,bʊk,ʊ,36.0
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,3,1_3,15_0_1_3,15_0_1,1,bʊk,k,13.0
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,0,2,1,2_1,15_0_2_1,15_0_2,0,bɔl,b,5.0
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,0,2,2,2_2,15_0_2_2,15_0_2,0,bɔl,ɔ,30.0


In [29]:
#Get the phonemes that we couldn't fix using the script
na_df2 = df3[df3['Phoneme_ID'].isna()]
na_df2.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID
784,GO,rPPA,1,WAB Naming,1,Baseline,paper clip,P EY P AXR K L IH P,2567.047161,2567.495228,...,0,66,5,13_5,1_0_66_5,1_0_66,1,pepɚklɪp,NaN,NaN
1151,GO,rPPA,1,WAB Naming,1,12,tape,T EY K,2608.391279,2608.747051,...,1,73,2,18_2,1_1_73_2,1_1_73,0,tek,NaN,NaN
1368,GO,rPPA,1,WAB Naming,2,12,matches,M EH T,2181.913849,2182.283240,...,2,55,2,20_2,1_2_55_2,1_2_55,0,mɛt,NaN,NaN
1424,BW,rPPA,8,WAB Naming,1,Baseline,screwdriver,S K R UW D R AY V AXR,1254.863336,1255.452862,...,0,12,1,11_1,8_0_12_1,8_0_12,0,skrudrɐvɚ,NaN,NaN
1439,BW,rPPA,8,WAB Naming,1,Baseline,paper clip,P EY P AXR K L IH P,1266.730034,1267.510267,...,0,14,6,13_6,8_0_14_6,8_0_14,0,pepɚklɪp,NaN,NaN


In [30]:
#define word identification codes for the phonemes that you want to drop (will need to drop the whole word)
values = ['1_0_66', '1_1_73', '1_2_55', '8_0_12', '8_0_14', '8_0_17', '8_1_11', '8_1_13']

#drop rows that contain any value in the list
df3 = df3[df3.Word_Sess_Code.isin(values) == False]

df3['Phoneme_ID'] = df3['Phoneme_ID'].astype('int')

df3.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Session_ID,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,1,1_1,15_0_1_1,15_0_1,0,bʊk,b,5
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,2,1_2,15_0_1_2,15_0_1,0,bʊk,ʊ,36
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,0,1,3,1_3,15_0_1_3,15_0_1,1,bʊk,k,13
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,0,2,1,2_1,15_0_2_1,15_0_2,0,bɔl,b,5
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,0,2,2,2_2,15_0_2_2,15_0_2,0,bɔl,ɔ,30


In [31]:
#This is to prep the data for some of the phonological processes

#Create a column to identify the phoneme that occurred before the produced phoneme
df3['Prod_Prev_Phon'] = (
    df3['Phoneme_ID']
    .shift()
    .where(
        df3['Word_Sess_Code'].shift() == df3['Word_Sess_Code'], 0
        )
    .astype('int')
    )

df3.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Prod_Word_N,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID,Prod_Prev_Phon
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,1,1_1,15_0_1_1,15_0_1,0,bʊk,b,5,0
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,2,1_2,15_0_1_2,15_0_1,0,bʊk,ʊ,36,5
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,3,1_3,15_0_1_3,15_0_1,1,bʊk,k,13,36
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,2,1,2_1,15_0_2_1,15_0_2,0,bɔl,b,5,0
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,2,2,2_2,15_0_2_2,15_0_2,0,bɔl,ɔ,30,5


In [32]:
#Create a column to identify the phoneme that occurred after the produced phoneme
df3['Prod_Next_Phon'] = (
    df3['Phoneme_ID']
    .shift(-1)
    .where(
        df3['Word_Sess_Code'].shift(-1) == df3['Word_Sess_Code'], 0
        )
    .astype('int')
    )

df3.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Prod_Phon_N,Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID,Prod_Prev_Phon,Prod_Next_Phon
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,1_1,15_0_1_1,15_0_1,0,bʊk,b,5,0,36
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,2,1_2,15_0_1_2,15_0_1,0,bʊk,ʊ,36,5,13
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,3,1_3,15_0_1_3,15_0_1,1,bʊk,k,13,36,0
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,1,2_1,15_0_2_1,15_0_2,0,bɔl,b,5,0,30
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,2,2_2,15_0_2_2,15_0_2,0,bɔl,ɔ,30,5,14


In [33]:
#Add in word information file for target items
word_info = (
    pd.read_csv("Resources/word_info.csv")
)

#Drop unneeded columns and rename other columns
word_info = (
    word_info
    .drop(columns=['N_Tot_Words.1','Target','Target_Word_Struc','Target_Syll_Struc'])
    .rename(columns={
        'Arpabet':'Target_Arpabet',
        'N_Tot_Syllables':'Target_N_Tot_Syllables',
        'N_Tot_Target_Phonemes':'Target_N_Tot_Phonemes',
        'N_Tot_Words':'Target_N_Tot_Words'
        })
)

word_info.head()

,Target_Arpabet,Word_ID,Target_N_Tot_Words,Target_IPA,Target_N_Tot_Syllables,Target_N_Tot_Phonemes
0,B UH K,1,1,bʊk,1,3
1,B AO L,2,1,bɔl,1,3
2,N AY F,3,1,nɐf,1,3
3,K AH P,4,1,kʌp,1,3
4,S EY F T IY P IH N,5,2,seftipɪn,3,8


In [34]:
# Merge word information with the main dataset
df4 = pd.merge(df3, word_info, on='Word_ID', how='left')
df4.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Production_IPA,IPA_singles,Phoneme_ID,Prod_Prev_Phon,Prod_Next_Phon,Target_Arpabet,Target_N_Tot_Words,Target_IPA,Target_N_Tot_Syllables,Target_N_Tot_Phonemes
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,bʊk,b,5,0,36,B UH K,1,bʊk,1,3
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,bʊk,ʊ,36,5,13,B UH K,1,bʊk,1,3
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,bʊk,k,13,36,0,B UH K,1,bʊk,1,3
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,bɔl,b,5,0,30,B AO L,1,bɔl,1,3
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,bɔl,ɔ,30,5,14,B AO L,1,bɔl,1,3


In [35]:
#Load the phonological information for the target items
word_phon_info = (
    pd.read_csv("Resources/word_phon_info.csv")
)

#Drop unneeded columns and rename other columns
word_phon_info = (
    word_phon_info
    .drop(columns=[
        'Word',
        'Word_ID',
        'Production_N'
        ])
    .rename(columns={
        'Phoneme':'Target_Phon_Arpabet',
        'Phoneme_ID':'Target_Phoneme_ID',
        'Word_NumID':'Target_Word_NumID',
        'Con_Cluster':'Target_Con_Cluster',
        'Clust_ID':'Target_Clust_ID',
        'Clus_Type':'Target_Clus_Type',
        'Clust_Phon_Pos':'Target_Clust_Phon_Pos',
        'Clust_Phon_Env':'Target_Clust_Phon_Env',
        'PhonClus_Syll_Env':'Target_Syll_Env',
        'Phoneme_Pos_Word':'Target_Word_Pos'
        })
)

#Get target IPA symbols
word_phon_info["Target_IPA"] = (
    # trans is a series, so use string accessor to split value strings
    word_phon_info["Target_Phon_Arpabet"]
    # perform the lookup in the dictionary of each individual value
    .apply(lambda v: dictionary.loc[v])
)

word_phon_info.head()

,Target_Phon_Arpabet,Code,Target_Phoneme_ID,Target_Syll_Env,Target_Word_Pos,Syllable_NumID,Target_Word_NumID,Target_Con_Cluster,Target_Clust_ID,Target_Clus_Type,Target_Clust_Phon_Pos,Target_Clust_Phon_Env,Target_IPA
0,B,1_1,5,#_V,#_V,1,1,0,NaN,NaN,NaN,NaN,b
1,UH,1_2,36,C_C,C_C,1,1,0,NaN,NaN,NaN,NaN,ʊ
2,K,1_3,13,V_#,V_#,1,1,0,NaN,NaN,NaN,NaN,k
3,B,2_1,5,#_V,#_V,1,1,0,NaN,NaN,NaN,NaN,b
4,AO,2_2,30,C_C,C_C,1,1,0,NaN,NaN,NaN,NaN,ɔ


In [36]:
# Merge target phonological information with the main dataset
df5 = pd.merge(df4, word_phon_info, on='Code', how='left')
df5.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Target_Syll_Env,Target_Word_Pos,Syllable_NumID,Target_Word_NumID,Target_Con_Cluster,Target_Clust_ID,Target_Clus_Type,Target_Clust_Phon_Pos,Target_Clust_Phon_Env,Target_IPA_y
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,#_V,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,C_C,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ʊ
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,V_#,V_#,1.0,1.0,0.0,NaN,NaN,NaN,NaN,k
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,#_V,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,C_C,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ɔ


In [37]:
#Clean up main dataset's columns and rename things
df6 = (
    df5
    .drop(columns=[
        'PRODUCTION_START',
        'PRODUCTION_END',
        'PHONEME_START',
        'PHONEME_END'
        ])
    .rename(columns={
        'Phoneme':'Prod_Arpabet',
        'IPA_singles':'Prod_Phon_IPA',
        'Production_IPA':'Prod_Word_IPA',
        'Phoneme_ID':'Prod_Phoneme_ID',
        'Target_IPA_x':'Target_Word_IPA',
        'Target_IPA_y':'Target_Phon_IPA',
        'PRODUCTION_DURATION':'Prod_Word_Dur',
        'PHONEME_DURATION':'Prod_Phon_Dur'
        })
)

#Fill in NAs with integer or string values, depending on column
df6['Target_Phoneme_ID'] = df6['Target_Phoneme_ID'].fillna(44).astype('int')
df6['Target_Syll_Env'] = df6['Target_Syll_Env'].fillna('addition').astype('str')
df6['Target_Word_Pos'] = df6['Target_Word_Pos'].fillna('addition').astype('str')

# Export clearned it file as a CSV, without the Pandas index, but with the header
df6.to_csv("Resources/all_data_cleaned.csv", index=False, header=True)
df6.head()



,RA,Project,PID,Test,Arm,Week,Target,Production,Prod_Word_Dur,Prod_Arpabet,...,Target_Syll_Env,Target_Word_Pos,Syllable_NumID,Target_Word_NumID,Target_Con_Cluster,Target_Clust_ID,Target_Clus_Type,Target_Clust_Phon_Pos,Target_Clust_Phon_Env,Target_Phon_IPA
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,B,...,#_V,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,UH,...,C_C,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ʊ
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,K,...,V_#,V_#,1.0,1.0,0.0,NaN,NaN,NaN,NaN,k
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,0.397365,B,...,#_V,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,0.397365,AO,...,C_C,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ɔ


In [38]:
#Create a column to identify the phoneme that was supposed to occur before the target phoneme
df6['Target_Prev_Phon'] = (
    df6['Target_Phoneme_ID']
    .shift()
    .where(
        df6['Word_Sess_Code'].shift() == df6['Word_Sess_Code'], 0
        )
    .astype('int')
    )

df6.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,Prod_Word_Dur,Prod_Arpabet,...,Target_Word_Pos,Syllable_NumID,Target_Word_NumID,Target_Con_Cluster,Target_Clust_ID,Target_Clus_Type,Target_Clust_Phon_Pos,Target_Clust_Phon_Env,Target_Phon_IPA,Target_Prev_Phon
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,B,...,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b,0
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,UH,...,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ʊ,5
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,K,...,V_#,1.0,1.0,0.0,NaN,NaN,NaN,NaN,k,36
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,0.397365,B,...,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b,0
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,0.397365,AO,...,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ɔ,5


In [39]:
#Create a column to identify the phoneme that was supposed to occur after the target phoneme
df6['Target_Next_Phon'] = (
    df6['Target_Phoneme_ID']
    .shift(-1)
    .where(
        df6['Word_Sess_Code'].shift(-1) == df6['Word_Sess_Code'], 0
        )
    .astype('int')
    )

df6.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,Prod_Word_Dur,Prod_Arpabet,...,Syllable_NumID,Target_Word_NumID,Target_Con_Cluster,Target_Clust_ID,Target_Clus_Type,Target_Clust_Phon_Pos,Target_Clust_Phon_Env,Target_Phon_IPA,Target_Prev_Phon,Target_Next_Phon
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,B,...,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b,0,36
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,UH,...,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ʊ,5,13
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,0.295646,K,...,1.0,1.0,0.0,NaN,NaN,NaN,NaN,k,36,0
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,0.397365,B,...,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b,0,30
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,0.397365,AO,...,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ɔ,5,14


In [40]:
#Assign target phonetic features
target_phon_dist_features = (
    phon_dist_features
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Target_")
)
target_phon_dist_features.head()

,Target_Phoneme_ID,Target_syllabic,Target_consonantal,Target_sonorant,Target_continuant,Target_delayed release,Target_approximant,Target_tap,Target_nasal,Target_voice,...,Target_velar,Target_alveolar,Target_post-alveolar,Target_dental,Target_palatal,Target_glottal,Target_stop,Target_fricative,Target_affricate,Target_glide
0,1,-1,-1,-1,1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,-1,1,-1,-1
1,2,-1,-1,1,1,0,1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,4,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,-1,-1,1
4,5,-1,1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1


In [41]:
#Assign production phonetic features
prod_phon_dist_features = (
    phon_dist_features
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Prod_")
)
prod_phon_dist_features.head()

,Prod_Phoneme_ID,Prod_syllabic,Prod_consonantal,Prod_sonorant,Prod_continuant,Prod_delayed release,Prod_approximant,Prod_tap,Prod_nasal,Prod_voice,...,Prod_velar,Prod_alveolar,Prod_post-alveolar,Prod_dental,Prod_palatal,Prod_glottal,Prod_stop,Prod_fricative,Prod_affricate,Prod_glide
0,1,-1,-1,-1,1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,-1,1,-1,-1
1,2,-1,-1,1,1,0,1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,4,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,-1,-1,1
4,5,-1,1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1


In [42]:
#Load place-voicing-manner information
phon_pvm = pd.read_csv("Resources/phon_pvm.csv")
phon_pvm.head()

,IPA_singles,Arpabet,Phoneme_ID,Place,Manner,Place_N,Manner_N,Height,Frontness,Height_N,Frontness_N
0,h,H,1,glottal,fricative,8,4,consonant,consonant,0,0
1,r,R,2,alveolar,retroflex approximant,4,6,consonant,consonant,0,0
2,w,W,3,bilabial,glide,1,8,consonant,consonant,0,0
3,j,Y,4,palatal,glide,6,8,consonant,consonant,0,0
4,b,B,5,bilabial,stop,1,1,consonant,consonant,0,0


In [43]:
#Assign pvm features for productions
prod_phon_pvm = (
    phon_pvm
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Prod_")
)
prod_phon_pvm.head()

,Prod_Phoneme_ID,Prod_Place,Prod_Manner,Prod_Place_N,Prod_Manner_N,Prod_Height,Prod_Frontness,Prod_Height_N,Prod_Frontness_N
0,1,glottal,fricative,8,4,consonant,consonant,0,0
1,2,alveolar,retroflex approximant,4,6,consonant,consonant,0,0
2,3,bilabial,glide,1,8,consonant,consonant,0,0
3,4,palatal,glide,6,8,consonant,consonant,0,0
4,5,bilabial,stop,1,1,consonant,consonant,0,0


In [44]:
#Assign pvm features for targets
target_phon_pvm = (
    phon_pvm
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Target_")
)
target_phon_pvm.head()

,Target_Phoneme_ID,Target_Place,Target_Manner,Target_Place_N,Target_Manner_N,Target_Height,Target_Frontness,Target_Height_N,Target_Frontness_N
0,1,glottal,fricative,8,4,consonant,consonant,0,0
1,2,alveolar,retroflex approximant,4,6,consonant,consonant,0,0
2,3,bilabial,glide,1,8,consonant,consonant,0,0
3,4,palatal,glide,6,8,consonant,consonant,0,0
4,5,bilabial,stop,1,1,consonant,consonant,0,0


In [45]:
#Merge all the feature information with the main dataset
df7 = (
    df6.merge(
            prod_phon_dist_features, 
            on='Prod_Phoneme_ID', 
            how='left')
        .merge(
            prod_phon_pvm, 
            on='Prod_Phoneme_ID', 
            how='left')
        .merge( 
            target_phon_dist_features, 
            on='Target_Phoneme_ID', 
            how='left')
        .merge( 
            target_phon_pvm, 
            on='Target_Phoneme_ID', 
            how='left')
        
        )
df7.columns.tolist()

['RA',
 'Project',
 'PID',
 'Test',
 'Arm',
 'Week',
 'Target',
 'Production',
 'Prod_Word_Dur',
 'Prod_Arpabet',
 'Prod_Phon_Dur',
 'NOTES',
 'Word_ID',
 'Session_ID',
 'Prod_Word_N',
 'Prod_Phon_N',
 'Code',
 'Phon_Sess_Code',
 'Word_Sess_Code',
 'Prod_Last_Phon',
 'Prod_Word_IPA',
 'Prod_Phon_IPA',
 'Prod_Phoneme_ID',
 'Prod_Prev_Phon',
 'Prod_Next_Phon',
 'Target_Arpabet',
 'Target_N_Tot_Words',
 'Target_Word_IPA',
 'Target_N_Tot_Syllables',
 'Target_N_Tot_Phonemes',
 'Target_Phon_Arpabet',
 'Target_Phoneme_ID',
 'Target_Syll_Env',
 'Target_Word_Pos',
 'Syllable_NumID',
 'Target_Word_NumID',
 'Target_Con_Cluster',
 'Target_Clust_ID',
 'Target_Clus_Type',
 'Target_Clust_Phon_Pos',
 'Target_Clust_Phon_Env',
 'Target_Phon_IPA',
 'Target_Prev_Phon',
 'Target_Next_Phon',
 'Prod_syllabic',
 'Prod_consonantal',
 'Prod_sonorant',
 'Prod_continuant',
 'Prod_delayed release',
 'Prod_approximant',
 'Prod_tap',
 'Prod_nasal',
 'Prod_voice',
 'Prod_spread gl',
 'Prod_constr gl',
 'Prod_labial',

In [46]:
#Drop columns not needed for pvm analysis
pvm_all_data = (
    df7
    .drop(columns=[
        'RA',
        'Project',
        'Test',
        'Arm',
        'Week'
        ])
)
pvm_all_data.head()

,PID,Target,Production,Prod_Word_Dur,Prod_Arpabet,Prod_Phon_Dur,NOTES,Word_ID,Session_ID,Prod_Word_N,...,Target_affricate,Target_glide,Target_Place,Target_Manner,Target_Place_N,Target_Manner_N,Target_Height,Target_Frontness,Target_Height_N,Target_Frontness_N
0,15,book,B UH K,0.295646,B,0.024363,Article (É) before word,1,0,1,...,-1,-1,bilabial,stop,1,1,consonant,consonant,0,0
1,15,book,B UH K,0.295646,UH,0.163408,Article (É) before word,1,0,1,...,0,0,vowel,vowel,0,0,high,back,1,3
2,15,book,B UH K,0.295646,K,0.107875,Article (É) before word,1,0,1,...,-1,-1,velar,stop,7,1,consonant,consonant,0,0
3,15,ball,B AO L,0.397365,B,0.014197,NaN,2,0,2,...,-1,-1,bilabial,stop,1,1,consonant,consonant,0,0
4,15,ball,B AO L,0.397365,AO,0.211006,NaN,2,0,2,...,0,0,vowel,vowel,0,0,mid-low,back,4,3


In [47]:
#Replace problem phonemes with correct ARPABET code or to a more readable format
pvm_all_data['Target_Word_IPA'] = (
    pvm_all_data['Target_Word_IPA']
    .replace(
            {
            'ɪresɚ':'əresɚ'
            },
        regex=True)
    .str.strip() #gets rid of white space at the beginning and ending of string
)

In [48]:
#Save as pvm_all_data.csv
pvm_all_data.to_csv("Resources/pvm_all_data.csv", index=False, header=True)